In [1]:
%matplotlib inline
import tensorflow as tf
from vahun.corpus import TSV_Corpus as Corpus
import numpy as np
from vahun.tools import Timer
from vahun.tools import explog
#from vahun.autoencoder import Autoencoder_ffnn
from vahun.tools import show_performance
from vahun.genetic import Settings
from vahun.tools import get_reconstruction

from vahun.Autoencoder_FFNN import Autoencoder_FFNN
from vahun.Autoencoder_Variational import Autoencoder_Variational
timer=Timer()



corpus_path='/mnt/store/velkey/mnsz2/brain_split.200k.maxlen20'
encode=400
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Xcorpus=Corpus(corpus_path=corpus_path,col=0,size=0)

Ycorpus=Corpus(corpus_path=corpus_path,col=1,size=0)

logger=explog(encoder_type="demo_autoencoder_splitbrain"+str(encode),
              encoding_dim=encode,
              feature_len=20,
              lang=corpus_path,
              unique_words=len(set(Xcorpus.wordlist)),
              name="demo_autoencoder_top_splitbrain"+str(encode),
              population_size=0,
              words=len(Xcorpus.wordlist))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [2]:
encoder=Autoencoder_FFNN(
                 logger=logger,tf_session=sess,
                 inputdim=len(Xcorpus.abc)*20,
                 layerlist=[encode,len(Xcorpus.abc)*20],
                 encode_index=1,corpus=Xcorpus,
                 optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                 nonlinear=tf.sigmoid,disp_step=100,
                 charnum=len(Xcorpus.abc))

In [3]:
encoder.train(Xcorpus.x_train,Xcorpus.x_valid,Xcorpus.x_test,
              512,30,
              Ycorpus.x_train,Ycorpus.x_valid,Ycorpus.x_test)

memoryerror during train reconstruction,using only the half data!


In [6]:
result=encoder.get_reconstruction(Xcorpus.x_test,Xcorpus,Ycorpus.x_test)

['           t_l_a_a_t', '           _ú_h_l_d_', '           _a_h_t_t_', '           túlhaladt', '           talhatatt', 3]
['           _ú_h_l_d_', '           t_l_a_a_t', '           s_l_a_a_ó', '           túlhaladt', '           súlhaladó', 2]
['        ú_d_n_á_á_a_', '        _j_o_s_g_n_k', '        _l_o_t_s_b_n', '        újdonságának', '        úldontásában', 5]
['        _j_o_s_g_n_k', '        ú_d_n_á_á_a_', '        a_k_s_á_á_a_', '        újdonságának', '        ajkosságának', 3]
['               a_k_s', '               _k_u_', '               _i_o_', '               akkus', '               aikos', 2]
['               _k_u_', '               a_k_s', '               o_j_k', '               akkus', '               okjuk', 3]
['          t_k_z_s_a_', '          _o_o_á_b_n', '          _a_o_á_s_l', '          tokozásban', '          takozással', 3]
['          _o_o_á_b_n', '          t_k_z_s_a_', '          k_r_l_s_a_', '          tokozásban', '          korolásban', 3]
['      

In [7]:
with open('/mnt/store/velkey/splitbrain_200_encoding', "a") as myfile:
    for it in result:
        string=""
        for i in it:
            string+=str(i)
            string+='\t'
        string+='\n'
        myfile.write(string)

In [7]:
stds=show_performance(encoder,Xcorpus.x_test,Xcorpus,20,printer=True,inputdepth=20,inputfsize=len(Xcorpus.abc))

           t_l_a_a_t 	            _a_h_t_t_ 	 5
           _ú_h_l_d_ 	            s_l_a_e_ó 	 6
        ú_d_n_á_á_a_ 	         _l_o_t_s_b_l 	 7
        _j_o_s_g_n_k 	         a_e_s_á_á_a_ 	 7
               a_k_s 	                _i_á_ 	 4
               _k_u_ 	                o_j_k 	 4
          t_k_z_s_a_ 	           _a_s_á_b_l 	 6
          _o_o_á_b_n 	           k_r_l_s_a_ 	 6
          k_z_h_t_e_ 	           _s_e_e_t_k 	 6
          _e_d_e_t_m 	           k_n_t_t_e_ 	 6
            v_s_r_a_ 	             _a_z_b_k 	 5
            _á_á_n_k 	             t_l_s_a_ 	 5
       j_r_a_-_e_n_k 	        _e_e_t_t_k_e_ 	 8
       _á_n_k_k_l_e_ 	        k_l_a_e_e_t_k 	 8
            ö_v_n_b_ 	             _l_e_y_e 	 5
            _r_é_y_e 	             e_m_n_r_ 	 5
           m_g_é_n_l 	            _e_y_s_e_ 	 6
           _e_y_k_é_ 	            m_n_e_t_l 	 6
               c_i_i 	                _i_n_ 	 3
               _h_l_ 	                t_e_ó 	 4


AttributeError: 'Autoencoder_FFNN' object has no attribute 'char_accuracy'

In [10]:
std=show_performance(encoder,["eh","kecske","kutya","aytuk","macska","árvíztűrő","fúró","kacsa","a","és"],Xcorpus,printer=True,inputfsize=len(Xcorpus.abc))

ValueError: cannot reshape array of size 980 into shape (10,49)

In [ ]:
def decode_critical(lista,enc=180):
    for POS in lista:
        encoded=np.ones(enc)*-10
        encoded[POS]=10
        a=encoder.decode([encoded])[0].reshape([10,len(corpus.abc)])
        b=corp.defeaturize_data_charlevel_onehot([a])
        print(b)

In [ ]:
import heapq
topstd=heapq.nlargest(6, range(len(stds)), stds.__getitem__)
big_stuff=[]
for i in range(50):
    big_stuff.append([])
    for topind in topstd:
        a=encoder.encode([corpus.x_test[i]])
        a[0][topind]=2
        b=encoder.decode([a[0]])[0].reshape([10,len(corpus.abc)])
        c=corpus.defeaturize_data_charlevel_onehot([b])
        big_stuff[i].append(c)
for row in big_stuff:
    print(row)